In [1]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [2]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []
location_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
    location_list.append(process.location)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list, location_list)),
                               columns=['name', 'id', 'location'])
# processes_df

In [3]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_process_df = processes_df[processes_df['name'].str.contains(r'heat production, heavy fuel oil, at industrial(?!$)')] # TODO: change keywords
search_process_df.reset_index(drop=True, inplace=True)
search_process_df

,name,id,location
0,"heat production, heavy fuel oil, at industrial...",4f582f90-d0ae-3552-be43-70c730e82af0,RoW
1,"heat production, heavy fuel oil, at industrial...",aa500328-2d77-33da-b481-c41d3c3eb020,Europe without Switzerland
2,"heat production, heavy fuel oil, at industrial...",e38883c8-7b6a-3426-b85e-2e3a05814350,CA-QC
3,"heat production, heavy fuel oil, at industrial...",c07848f3-adb4-3066-af1c-3b48ef7e02be,CH


In [24]:
test_process = client.get(olca.Process, uid='4f582f90-d0ae-3552-be43-70c730e82af0')

In [39]:
test_exchange = test_process.exchanges[0]

In [41]:
test_exchange.amount

1.0

In [42]:
test_amount = []

In [43]:
test_amount.append(test_exchange.amount)

In [45]:
test_exchange.amount = 0.5

In [47]:
test_amount.append(test_exchange.amount)

In [49]:
test_process

Process(id='4f582f90-d0ae-3552-be43-70c730e82af0', olca_type='Process', name='heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U', description='Inventory for heat supply.;[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes described in the ecoinvent version 3 change report (http://www.ecoinvent.org/database/ecoinvent-version-3/reports-of-changes/), and the results of the central updates were reviewed extensively. The changes added e.g. consistent water flows and other information throughout the database. The documentation of this dataset can be found in the ecoinvent reports of version 2, which are still available via the ecoinvent website. The change report linked above covers all central changes that were made during the conversion process.]  Heat delivered by the furnace not including losses during distribution in the factory.', version='03.119.004', last_change='2021-01-04T12:23:36.342+01:00', category=Ref(id='157ef931-ad38-378c-bfdb-2585cf1f2c6a', olca_type='Category', name='3530a: Steam and air conditioning supply based on liquid fuels ', description=None, version=None, last_change=None, category_path=['D:Electricity, gas, steam and air conditioning supply', '35:Electricity, gas, steam and air conditioning supply', '353:Steam and air conditioning supply', '3530:Steam and air conditioning supply'], library=None, ref_unit=None, location=None, flow_type=None, process_type=None), tags=None, library=None, allocation_factors=None, default_allocation_method=None, exchanges=[Exchange(id=None, olca_type='Exchange', avoided_product=False, cost_formula=None, cost_value=0.0106, currency=Ref(id='7de74e65-ff97-404f-bf57-5d2a9134fb43', olca_type='Currency', name='Euro', description=None, version=None, last_change=None, category_path=None, library=None, ref_unit=None, location=None, flow_type=None, process_type=None), internal_id=2, flow=Ref(id='71e2f1db-a2c5-44d0-8337-dfff15be974d', olca_type='Flow', name='heat, district or industrial, other than natural gas', description=None, version=None, last_change=None, category_path=['D:Electricity, gas, steam and air conditioning supply', '35:Electricity, gas, steam and air conditioning supply', '351:Electric power generation, transmission and distribution', '3510:Electric power generation, transmission and distribution'], library=None, ref_unit='MJ', location=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, process_type=None), flow_property=Ref(id='f6811440-ee37-11de-8a39-0800200c9a66', olca_type='FlowProperty', name='Energy', description=None, version=None, last_change=None, category_path=['Technical flow properties'], library=None, ref_unit=None, location=None, flow_type=None, process_type=None), input=False, quantitative_reference=True, base_uncertainty=None, default_provider=None, amount=0.5, amount_formula=None, unit=Ref(id='52765a6c-3896-43c2-b2f4-c679acf13efe', olca_type='Unit', name='MJ', description=None, version=None, last_change=None, category_path=None, library=None, ref_unit=None, location=None, flow_type=None, process_type=None), dq_entry=None, uncertainty=None, description='EcoSpold01Location=CH'), Exchange(id=None, olca_type='Exchange', avoided_product=False, cost_formula=None, cost_value=None, currency=None, internal_id=16, flow=Ref(id='2966d161-5281-47d6-86d3-efbb0a4b2110', olca_type='Flow', name='heavy fuel oil', description=None, version=None, last_change=None, category_path=['C:Manufacturing', '19:Manufacture of coke and refined petroleum products', '192:Manufacture of refined petroleum products', '1920:Manufacture of refined petroleu

In [21]:
process_list_all[0]

'heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U'

In [23]:
exchange_descriptor = client.get_descriptors(olca.Exchange)
exchange_list_all = []
for exchange in exchange_descriptor:
    exchange_list_all.append(exchange_descriptor.)


<generator object Client.get_descriptors at 0x7fe25038d660>

In [5]:
olca.flow_of(new_process)

TypeError: flow_of() missing 2 required positional arguments: 'flow_type' and 'flow_property'

In [23]:
olca.input_of(new_process)

TypeError: input_of() missing 1 required positional argument: 'flow'

In [ ]:
process_to_search_df = pd.read_excel('List_of_Materials_ecoinvent.xlsx', header=None)

In [ ]:
# # Search specific process databases with key word(s)
# # Note that the search is blankspace-sensitive
# def process_search(process_to_search_df):
#     search_process_list = []
#     for i in range(len(process_to_search_df)):
#         search_process_df = processes_df[processes_df['name'].str.contains(process_to_search_df.iloc[i, 0], regex=False)]# TODO: change keywords
#         search_process.append(search_process_df)
#     return search_process_list
# large_df = pd.concat(process_search(process_to_search_df), ignore_index=True)

In [ ]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
# flows_df

Note that the fields ``database`` and ``code`` are added automatically for us.

There are only a few methods for an activity. We can see the activity key (combination of database and code):

In [ ]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')] # TODO: change keywords
search_flow_df.reset_index(drop=True, inplace=True)

# search_flow_df

In [20]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
product_system_locations = pd.Series(search_process_df['location'])

# for i in range(len(search_list_processID)):
for i in range(2): # for test
    print('Creating product system of: '+search_process_df['name'][i]+' | '+product_system_locations[i])
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)
    # print('Remaining number of product systems to be created: '+str(len(search_list_processID)-i-1))
    print('Remaining number of product systems to be created: '+str(2-i-1)) # for test

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product_system_name', 'product_system_id'])

product_system_df.assign(product_system_location=product_system_locations)

Creating product system of: heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U | RoW
Remaining number of product systems to be created: 1
Creating product system of: heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U | Europe without Switzerland
Remaining number of product systems to be created: 0


,product_system_name,product_system_id,product_system_location
0,"heat production, heavy fuel oil, at industrial...",dafb09af-1dae-414a-bc34-7fec7b7b6277,RoW
1,"heat production, heavy fuel oil, at industrial...",0d651f5c-9b3c-484d-a0fe-9dfbf3a20995,Europe without Switzerland


In [15]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013') # TODO: choose impact method accordingly

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [16]:
# for ps in range(len(product_system_names)):
for ps in range(2): # for test
    
    print('Working on '+product_system_names[ps]+' | '+product_system_locations[ps]+' | '+product_system_ids[ps])

    setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[ps])

    print(setup.product_system.id)

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result/'+product_system_locations[ps]+'_'+'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(len(product_system_names)-ps-1))

Working on heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U | RoW | 1a7e3029-b37c-47b7-aa2d-cd93d774657a
1a7e3029-b37c-47b7-aa2d-cd93d774657a


ERROR:root:calculation failed: 500: Failed to call method public org.openlca.ipc.RpcResponse org.openlca.ipc.handlers.Calculator.calculate(org.openlca.ipc.RpcRequest): null
ERROR:root:Excel export to calc_result/RoW_calc_result.xlsx failed: 501: The Excel export is currently only implemented for calculation results


Remaining number of processes to be calculated: 1
Working on heat production, heavy fuel oil, at industrial furnace 1MW | heat, district or industrial, other than natural gas | Cutoff, U | Europe without Switzerland | fe437ddb-2072-4682-869a-052746bbdc9a
fe437ddb-2072-4682-869a-052746bbdc9a


ERROR:root:calculation failed: 500: Failed to call method public org.openlca.ipc.RpcResponse org.openlca.ipc.handlers.Calculator.calculate(org.openlca.ipc.RpcRequest): null
ERROR:root:Excel export to calc_result/Europe without Switzerland_calc_result.xlsx failed: 501: The Excel export is currently only implemented for calculation results


Remaining number of processes to be calculated: 0


In [ ]:
GWP100a_df = pd.DataFrame()
# for i in range(len(product_system_locations)):
for i in range(2): # for test
    filename_xlsx = 'calc_result/'+product_system_locations[i]+'_calc_result.xlsx'
    # filename_xlsx = 'calc_result/'+product_system_locations[0]+'_calc_result.xlsx' # for test
    results_db = pd.ExcelFile(filename_xlsx)
    impact_df = results_db.parse('Impacts')
    setup_df = results_db.parse('Calculation setup')

    new_header = impact_df.iloc[0] #grab the first row for the header
    impact_df = impact_df[1:] #take the data less the header row
    impact_df.columns = new_header #set the header row as the df header

    GWP100a_series = pd.Series(impact_df.iloc[2, 2:])
    GWP100a_series['Process'] = search_process_df['name'][i]
    GWP100a_series['location_extended'] = setup_df.iloc[3, 2]

    GWP100a_df[product_system_locations[i]] = GWP100a_series

GWP100a_df.T.to_excel('GWP100a_summary.xlsx')


In [ ]:
process